# 框架图

```mermaid
graph TB
    subgraph Foundation["基础层 Foundation Layer"]
        WTSMarcos["WTSMarcos.h<br/>基础宏定义"]
        WTSVersion["WTSVersion.h<br/>版本信息"]
        WTSTypes["WTSTypes.h<br/>基础类型定义"]
        WTSError["WTSError.hpp<br/>错误代码定义"]
        FasterDefs["FasterDefs.h<br/>性能优化定义"]
        LoaderDef["LoaderDef.hpp<br/>加载器定义"]
    end
    
    subgraph CoreData["核心数据层 Core Data Layer"]
        WTSObject["WTSObject.hpp<br/>基础对象类"]
        WTSVariant["WTSVariant.hpp<br/>变体数据类型"]
        WTSStruct["WTSStruct.h<br/>核心数据结构"]
        WTSCollection["WTSCollection.hpp<br/>数据集合"]
    end
    
    subgraph MarketData["市场数据层 Market Data Layer"]
        WTSDataDef["WTSDataDef.hpp<br/>市场数据定义"]
        WTSExpressData["WTSExpressData.hpp<br/>扩展数据"]
        WTSSessionInfo["WTSSessionInfo.hpp<br/>会话信息"]
        WTSContractInfo["WTSContractInfo.hpp<br/>合约信息"]
        WTSSwitchItem["WTSSwitchItem.hpp<br/>主力切换规则"]
    end
    
    subgraph Trading["交易层 Trading Layer"]
        WTSTradeDef["WTSTradeDef.hpp<br/>交易定义"]
        WTSRiskDef["WTSRiskDef.hpp<br/>风险定义"]
        ExecuteDefs["ExecuteDefs.h<br/>执行单元定义"]
        RiskMonDefs["RiskMonDefs.h<br/>风险监控定义"]
    end
    
    subgraph StrategyDefs["策略定义层 Strategy Definitions"]
        CtaStrategyDefs["CtaStrategyDefs.h<br/>CTA策略定义"]
        HftStrategyDefs["HftStrategyDefs.h<br/>HFT策略定义"]
        SelStrategyDefs["SelStrategyDefs.h<br/>SEL策略定义"]
        UftStrategyDefs["UftStrategyDefs.h<br/>UFT策略定义"]
    end
    
    subgraph StrategyCtx["策略上下文接口层 Strategy Context Interfaces"]
        ICtaStraCtx["ICtaStraCtx.h<br/>CTA策略上下文"]
        IHftStraCtx["IHftStraCtx.h<br/>HFT策略上下文"]
        ISelStraCtx["ISelStraCtx.h<br/>SEL策略上下文"]
        IUftStraCtx["IUftStraCtx.h<br/>UFT策略上下文"]
    end
    
    subgraph DataMgmt["数据管理接口层 Data Management Interfaces"]
        IBaseDataMgr["IBaseDataMgr.h<br/>基础数据管理"]
        IDataManager["IDataManager.h<br/>数据管理"]
        IDataFactory["IDataFactory.h<br/>数据工厂"]
        IDataReader["IDataReader.h<br/>数据读取"]
        IDataWriter["IDataWriter.h<br/>数据写入"]
        IBtDtReader["IBtDtReader.h<br/>回测数据读取"]
        IRdmDtReader["IRdmDtReader.h<br/>随机数据读取"]
    end
    
    subgraph APILayer["API接口层 API Interfaces"]
        IParserApi["IParserApi.h<br/>行情解析API"]
        ITraderApi["ITraderApi.h<br/>交易API"]
        ISessionMgr["ISessionMgr.h<br/>会话管理"]
        IHotMgr["IHotMgr.h<br/>热点管理"]
        ILogHandler["ILogHandler.h<br/>日志处理"]
        IExpFactory["IExpFactory.h<br/>扩展工厂"]
    end
    
    Foundation --> CoreData
    CoreData --> MarketData
    MarketData --> Trading
    Trading --> StrategyDefs
    StrategyDefs --> StrategyCtx
    
    Foundation --> DataMgmt
    Foundation --> APILayer
    CoreData --> StrategyCtx
    MarketData --> DataMgmt
    Trading --> APILayer
    
    WTSTypes --> WTSStruct
    WTSTypes --> WTSObject
    WTSObject --> WTSVariant
    WTSStruct --> WTSDataDef
    WTSStruct --> WTSContractInfo
    WTSContractInfo --> WTSSwitchItem
    
    CtaStrategyDefs --> ICtaStraCtx
    HftStrategyDefs --> IHftStraCtx
    SelStrategyDefs --> ISelStraCtx
    UftStrategyDefs --> IUftStraCtx
    
    IDataFactory --> IDataReader
    IDataFactory --> IDataWriter
    IBaseDataMgr --> IDataManager
```

# 基础层

## 宏定义 WTSMarcos.h

## 版本信息 WTSVersion.h

## 类型定义 WTSTypes.h

### 合约类型 tagContractCategory
```cpp
typedef enum tagContractCategory
{
	CC_Stock,			// 股票
	CC_Future,			// 期货
	CC_FutOption,		// 期货期权，商品期权属于这个分类
	CC_Combination,		// 组合
	CC_Spot,			// 即期
	CC_EFP,				// 期转现
	CC_SpotOption,		// 现货期权，股指期权属于这个分类
	CC_ETFOption,		// 个股期权，ETF期权属于这个分类

	CC_DC_Spot	= 20,	// 数字货币现货
	CC_DC_Swap,			// 数字货币永续
	CC_DC_Future,		// 数字货币期货
	CC_DC_Margin,		// 数字货币杠杆
	CC_DC_Option,		// 数字货币期权

	CC_UserIndex = 90	// 自定义指数
} ContractCategory;
```

|  | CC_Stock<br/>(股票) | CC_Future<br/>(期货) | CC_FutOption<br/>(期货期权) | CC_SpotOption<br/>(现货期权) | CC_ETFOption<br/>(ETF期权) | CC_Spot<br/>(现货) | CC_EFP<br/>(期转现) | CC_Combination<br/>(组合合约) |
|-------------|---------------------|----------------------|----------------------------|-----------------------------------|---------------------------|-------------------|---------------------|-------------------------------|
| **理解** |股份制公司的所有权份额| 到期"必须执行"的订单，事先确定好：<br/>商品、交货日期、价格、数量| 到期可以“自行决定是否执行”的订单。<br/>须花一点权利金 | 和 CC_FutOption类似| 和 CC_FutOption类似；<br/>但是交易的是ETF (Exchange Traded Fund, 交易所交易基金)份额  | 立即执行订单 | Exchange for Physicals<br/>私下按照商定价格换货，<br/>之后上报交易所登记 | 由多个单一合约组合而成 |
| **标的物** |  | 商品本身或现金结算 | 订单合约 | 商品本身或现金结算 | ETF (Exchange Traded Fund, 交易所交易基金)份额 |  |  | |
| **权利与义务** | 拥有所有权和分红权 | 双方都有义务在到期日履约 | 买方只有权利，卖方有义务 | 买方只有权利，卖方有义务 | 买方只有权利，卖方有义务 |  |  | |
| **到期日** | 无 | 有 | 有 | 有 | 有 | 无 | - | 取决于组合内的合约 |
| **现实世界例子** | `600519.SH`<br/>(贵州茅台) | `rb2410.SHFE`<br/>(螺纹钢期货) | `m2409-C-3500.DCE`<br/>(豆粕看涨期权) | `IO2409-C-4000.CFFEX`<br/>(沪深300股指看涨期权) | `10005231.SH`<br/>(上证50ETF期权) | 买卖股票、即期外汇交易 | | |

| 字段 | CC_DC_Spot<br/>(数字货币现货) | CC_DC_Future<br/>(数字货币期货) | CC_DC_Swap<br/>(数字货币永续合约) | CC_DC_Margin<br/>(数字货币杠杆) | CC_DC_Option<br/>(数字货币期权) |
|-------------|-------------------------------|----------------------------------|-----------------------------------|----------------------------------|----------------------------------|
| **核心定义** | 直接买卖数字货币资产本身的所有权。 | 在未来特定日期以约定价格买卖数字货币的标准化合约。 | 一种没有到期日的特殊期货<br/>定期根据永续价和现货价向多空双方定期付费/收费，使合约价格锚定现货价格| 在现货交易基础上，通过向平台借贷来放大交易规模的一种交易方式。 | 在未来以特定价格买入或卖出数字货币的权利，而非义务。 |
| **交易对象** | 真实资产的所有权<br/>(如一枚比特币) | 一份有期限的**未来合同** |  | 加了杠杆的真实资产 |  |
| **是否持有资产** | **是**，你完全拥有这枚数字货币。 | **否**，你只拥有一个合约头寸。 | **否**，你只拥有一个合约头寸。 | **是**，你拥有真实资产，但部分是借来的。 |  |
| **到期日** | **无** | **有** (如当季、次季)，到期后合约自动结算。 | **无**，这是其最核心的特点。 | **无** | **有** (如日权、周权、季权)。 |
| **核心机制** | 钱货两清，即时成交。 |  | 资金费率：通过多空双方定期付费/收费，使合约价格锚定现货价格。 | 借贷与利息：需要为借来的资金或币支付利息。 | 权利金：买方支付权利金获得权利，卖方收取权利金承担义务。 |
| **头寸延续方式** | **无需延续**，可永久持有。 |  | **自动延续**：只要保证金充足，头寸可一直持有，无需任何操作。 | **无需延续**，可永久持有（只要能支付利息且不爆仓）。 | **到期后权利失效** |
| **主要目的** | 价值投资、长期持有（屯币）、作为交易基础。 | 套期保值、基于明确时间预期的价格投机。 | 便捷地进行杠杆投机，是目前最主流的衍生品玩法。 | 在看好现货行情时，放大本金，博取更高收益。 | |
| **现实世界例子** | 在币安"币币交易"区，用USDT买入BTC。 | 交易 `BTCUSDT当季` (BTCUSDT Quarterly) 合约。 | 交易 `BTCUSDT永续` (BTCUSDT Perpetual) 合约。 | 在币安"杠杆交易"区，选择5倍杠杆买入BTC/USDT现货对。 | 交易 `BTC-241227-80000-C` (年底到期，行权价8万美元的BTC看涨期权)。 |

### 期权类型 tagOptionType
```cpp
typedef enum tagOptionType
{
	OT_None = 0,        // 无期权类型
	OT_Call = '1',		// 买入类型的期权（约定某个期限时以指定价格买入，希望买入价低于期限时的市场价，即看涨）
	OT_Put	= '2'		// 卖出类型的期权（约定某个期限时以指定价格卖出，希望卖出价高于期限时的市场价，即看跌）
} OptionType;
```

### 开/平仓类型 tagCoverMode
```cpp
typedef enum tagCoverMode
{
	CM_OpenCover,		// 开平：开仓和平仓
	CM_CoverToday,		// 开平昨平今：开仓、平昨仓、平今仓
	CM_UNFINISHED,		// 平未了结的：平掉未完成的持仓
	CM_None			// 不区分开平：不区分开仓和平仓
} CoverMode;
```
| 字段 | CM_CoverToday<br/>(开平昨平今) | CM_OpenCover<br/>(开平) | CM_None<br/>(不区分开平) | CM_UNFINISHED<br/>(平未了结) |
|------|--------------------------------|--------------------------|---------------------------|-------------------------------|
| **核心机制** | 严格区分昨仓和今仓 | 只区分开仓和平仓，不区分今昨 | 只关心净头寸的变化，没有开平概念 | 平掉未最终结算的特殊仓位 |
| **下单指令** | 开仓`Open`<br/>平今仓`CloseToday`<br/>平昨仓`CloseYesterday` | `Open`, `Close` | `Buy`, `Sell` | 特定业务指令 |
| **持仓视角** | `Position = YesterdayPos + TodayPos` | `Position = TotalPosition` | `Position = NetPosition` | 特殊持仓管理 |
| **主要适用市场** | 中国内地期货市场<br/>(SHFE, DCE, CZCE, CFFEX) | 国际期货市场、外盘期货 | 股票、数字货币市场 | 期权行权、特殊结算场景 |


### 交易模式 tagTradingMode
```cpp
typedef enum tagTradingMode
{
	TM_Both,	// 多空都支持：可以做多也可以做空（T+0）
	TM_Long,	// 只能做多：只允许做多方向（T+0）
	TM_LongT1,	// 只能做多且仅支持T+1：当天（T日）买入的头寸，最早要到下一个交易日（T+1日）才能卖出
	TM_None = 9	// 不能交易：该合约不允许交易
} TradingMode;
```

### 订单价格模式 `tagPriceMode`
```cpp
typedef enum tagPriceMode
{
	PM_Both,		// 市价限价都支持：既支持市价单也支持限价单
	PM_Limit,		// 只支持限价：只允许限价单
	PM_Market,		// 只支持市价：只允许市价单
	PM_None	= 9		// 不支持交易：该合约不支持交易
} PriceMode;
```

### K线数据类型 `tagKlineFieldType`
```cpp
typedef enum tagKlineFieldType
{
	KFT_OPEN,      // 开盘价字段
	KFT_HIGH,      // 最高价字段
	KFT_LOW,       // 最低价字段
	KFT_CLOSE,     // 收盘价字段
	KFT_DATE,      // 日期字段
	KFT_TIME,      // 时间字段
	KFT_VOLUME,    // 总成交量字段：正数或零
	KFT_SVOLUME    // 带正负号的成交量字段
} WTSKlineFieldType;
```

### K线周期类型 `tagKlinePeriod`
```cpp
typedef enum tagKlinePeriod
{
	KP_Tick,       // Tick级别：最小时间单位，即逐笔数据
	KP_Minute1,    // 1分钟K线
	KP_Minute5,    // 5分钟K线
	KP_DAY,        // 日K线
	KP_Week,       // 周K线
	KP_Month       // 月K线
} WTSKlinePeriod;
```

### 日志级别 `tagLogLevel`
```cpp
typedef enum tagLogLevel
{
	LL_ALL	= 100, // 所有日志：记录所有级别的日志
	LL_DEBUG,      // 调试日志：记录调试信息
	LL_INFO,       // 信息日志：记录一般信息
	LL_WARN,       // 警告日志：记录警告信息
	LL_ERROR,      // 错误日志：记录错误信息
	LL_FATAL,      // 致命错误：记录致命错误信息
	LL_NONE        // 无日志：不记录任何日志
} WTSLogLevel;
```

### 订单价格类型 `tagPriceType`
```cpp
typedef enum tagPriceType
{
	WPT_ANYPRICE	= 0,			// 市价单：以任意价格成交
	WPT_LIMITPRICE,					// 限价单：以指定价格或更好价格成交
	WPT_BESTPRICE,					// 最优价：以最优价格成交
	WPT_LASTPRICE,					// 最新价：以最新成交价成交

	//////////////////////////////////////////////////////////////////////////
	//以下对标CTP的价格类型
	WPT_CTP_LASTPLUSONETICKS = 20,	// 最新价+1ticks：CTP最新价加一个最小变动价位
	WPT_CTP_LASTPLUSTWOTICKS,		// 最新价+2ticks：CTP最新价加两个最小变动价位
	WPT_CTP_LASTPLUSTHREETICKS,		// 最新价+3ticks：CTP最新价加三个最小变动价位
	WPT_CTP_ASK1,					// 卖一价：CTP卖一价
	WPT_CTP_ASK1PLUSONETICKS,		// 卖一价+1ticks：CTP卖一价加一个最小变动价位
	WPT_CTP_ASK1PLUSTWOTICKS,		// 卖一价+2ticks：CTP卖一价加两个最小变动价位
	WPT_CTP_ASK1PLUSTHREETICKS,		// 卖一价+3ticks：CTP卖一价加三个最小变动价位
	WPT_CTP_BID1,					// 买一价：CTP买一价
	WPT_CTP_BID1PLUSONETICKS,		// 买一价+1ticks：CTP买一价加一个最小变动价位
	WPT_CTP_BID1PLUSTWOTICKS,		// 买一价+2ticks：CTP买一价加两个最小变动价位
	WPT_CTP_BID1PLUSTHREETICKS,		// 买一价+3ticks：CTP买一价加三个最小变动价位
	WPT_CTP_FIVELEVELPRICE,			// 五档价，中金所市价：CTP五档价格

	//////////////////////////////////////////////////////////////////////////
	//以下对标DC的价格类型
	WPT_DC_POSTONLY	= 100,			// 只做maker单：DC只做挂单方
	WPT_DC_FOK,						// 全部成交或立即取消：DC全部成交或立即取消
	WPT_DC_IOC,						// 立即成交并取消剩余：DC立即成交并取消剩余
	WPT_DC_OPTLIMITIOC				// 市价委托立即成交并取消剩余：DC市价委托立即成交并取消剩余
} WTSPriceType;
```

***CTP*** ：Comprehensive Transaction Platform (综合交易平台)，是由上海期货信息技术有限公司（上期技术）开发的期货交易系统接口的名称。

通用价格类型
- `WPT_ANYPRICE`：市价单，以当前市场上可以成交的最低价格价格立即成交的订单
  - 成交速度快，但成交价格不确定，可能出现 *滑点*
- `WPT_LIMITPRICE`：限价单，以指定价格或更优价格成交的订单
  - 如果滑点导致不符合指定价或更优，订单将会取消
- `WPT_BESTPRICE`：最优价，以当前盘口最优价为指定价格的限价单
  - ***买一价***：买入时，卖盘中最低价格；***卖一价***：卖出时，买盘中最高价格
- `WPT_LASTPRICE`：最新价，以最新的成交价作为指定价格的限价单

CTP 专有价格类型（限价单）
- 最新价
  - `WPT_CTP_LASTPLUSONETICKS`：最新价 +1 ticks（加一个最小变动价位）
  - `WPT_CTP_LASTPLUSTWOTICKS`：最新价 +2 ticks（加两个最小变动价位）
  - `WPT_CTP_LASTPLUSTHREETICKS`：最新价 +3 ticks（加三个最小变动价位）
- 卖一价
  - `WPT_CTP_ASK1`：卖一价
  - `WPT_CTP_ASK1PLUSONETICKS`：卖一价 +1 ticks（加一个最小变动价位）
  - `WPT_CTP_ASK1PLUSTWOTICKS`：卖一价 +2 ticks（加两个最小变动价位）
  - `WPT_CTP_ASK1PLUSTHREETICKS`：卖一价 +3 ticks（加三个最小变动价位）
- 买一价
  - `WPT_CTP_BID1`：买一价
  - `WPT_CTP_BID1PLUSONETICKS`：买一价 +1 ticks（加一个最小变动价位）
  - `WPT_CTP_BID1PLUSTWOTICKS`：买一价 +2 ticks（加两个最小变动价位）
  - `WPT_CTP_BID1PLUSTHREETICKS`：买一价 +3 ticks（加三个最小变动价位）
- 五档价
  - `WPT_CTP_FIVELEVELPRICE`：指定价为当前盘口最优的前五档价格
    - 例如买入时，将卖盘的价格从低到高排序，指定价即为其中的第五个价格

数字货币专有价格类型
- ***Maker-Taker 手续费模型***
  - 交易市场里有两类交易者 ***挂单Maker*** 和 ***吃单Taker***
    - Maker：将买/卖单以指定价格放到 *订单簿* 上
      - 下单，但是该单尚未与订单簿上的任何订单匹配成交
      - 手续费较低甚至返佣（因为增加了交易市场的深度）
    - Taker：买走订单簿上的订单
      - 下单，且该单与订单簿上的某订单匹配成交
      - 手续费较高（因为削减了交易市场的深度）

- `WPT_DC_POSTONLY`：限价单，只做挂单方
  - 核心是享受交易所提供的极低手续费，甚至是负费率（返佣）
- `WPT_DC_FOK`：限价吃单，能够立即成交的部分立即成交（吃单Taker），任何未能立即成交的剩余部分则立刻被取消
- `WPT_DC_IOC`：限价吃单，如果能立即、全部成交则立即成交（吃单Taker），否则就立即全部取消
- `WPT_DC_OPTLIMITIOC`：市价吃单，以市价形式发出所有订单，直到数量全部满足，或者订单簿被吃穿

### 订单生命周期 `tagTimeCondition`
```cpp
typedef enum tagTimeCondition
{
	WTC_IOC = '1',	// 立即完成（ Immediate-Or-Cancel）：订单中能够立即成交的部分立即成交，任何未能立即成交的剩余部分则立刻被交易所撤销
	WTC_GFS,	// 本节有效（Good-For-Session）：订单只在当前的交易小节（Session）内有效。当这个交易小节结束时，如果订单仍未成交，将被自动撤销
				// 有些交易市场分为多个交易小节：例如上午盘、下午盘
	WTC_GFD,	// 当日有效（Good-For-Day）：当日有效，即订单从提交开始，直到当天收盘为止都有效
} WTSTimeCondition;
```

### `tagOrderFlag`
```cpp
typedef enum tagOrderFlag
{
	WOF_NOR = '0',		// 普通订单：如果一次成交不完，未成交的部分会继续留在订单簿中排队等待，直到全部成交
	WOF_FAK,			// FAK（Fill and Kill）：订单中能够立即成交的部分立即成交，任何未能立即成交的剩余部分则立刻被交易所撤销
	WOF_FOK,			// FOK（Fill or Kill）：订单必须在瞬间、完全地成交，否则就立即全部取消。不允许任何部分成交
} WTSOrderFlag;
```

### 开/平方向类型 `tagOffsetType`
```cpp
typedef enum tagOffsetType
{
	WOT_OPEN			= '0',	// 开仓：建立新的持仓
	WOT_CLOSE,					// 平仓
	WOT_FORCECLOSE,				// 强平：强制平仓
	WOT_CLOSETODAY,				// 平今：平今日建立的持仓
	WOT_CLOSEYESTERDAY,			// 平昨：平昨日建立的持仓
} WTSOffsetType;
```

### 交易方向类型 `tagDirectionType`
```cpp
/*
 *	多空方向枚举
 *	定义了交易的多空方向
 */
typedef enum tagDirectionType
{
	WDT_LONG			= '0',	// 做多：买入做多
	WDT_SHORT,					// 做空：卖出做空
	WDT_NET						// 净：净持仓方向
} WTSDirectionType;
```
| 字段 | WDT_LONG (做多) | WDT_SHORT (做空) | WDT_NET (净头寸) |
|------|------------------|------------------|------------------|
| 核心含义 | 买入持有，期望价格上涨 | 卖出持有，期望价格下跌 | 多头与空头合并计算后的结果 |
| 角色 | 持仓方向 | 持仓方向 | 统计/查询方式 |
| 数值表示 | 正数 (e.g., +10) | 负数 (e.g., -10) | 可正可负可零 |
| 操作 | 买入开仓 (Buy Open) | 卖出开仓 (Sell Open) | - |
| 通俗比喻 | 买房等升值 | 借房卖掉等降价再买回 | 计算你名下所有房产的净值 |
| 主要用途 | 建立看涨头寸 | 建立看跌头寸 | 评估总体风险敞口、宏观持仓统计 |

### 业务类型 `tagBusinessType`
```cpp
/*
 *	业务类型枚举
 *	定义了交易业务的不同类型
 */
typedef enum tagBusinessType
{
	BT_CASH		= '0',	// 普通买卖：现金交易
	BT_ETF		= '1',	// ETF申赎：ETF申购赎回
	BT_EXECUTE	= '2',	// 期权行权：期权执行
	BT_QUOTE	= '3',	// 期权报价：期权报价
	BT_FORQUOTE = '4',	// 期权询价：期权询价
	BT_FREEZE	= '5',	// 期权对锁：期权对锁
	BT_CREDIT	= '6',	// 融资融券：信用交易
	BT_UNKNOWN			// 未知业务类型：未识别的业务类型
} WTSBusinessType;
```
- `BT_CASH` (普通买卖 / Cash Trading)：用资金卖入或直接卖出金融资产
- `BT_ETF` (ETF申购或赎回 / ETF Creation and Redemption)：一级市场的ETF申购与赎回操作，而不是二级市场的买卖。
  - **一级市场**：也称为发行市场，是资金需求方（如公司、政府）首次向资金供给方（投资者）出售全新金融证券（如股票、债券）的市场。
  - **二级市场**：也称为流通市场，是已经发行的证券在不同投资者之间进行买卖交易的市场。
  - **申购**：投资者按照基金公司公布的 *申购赎回清单*，在二级市场上买齐所需的一篮子股票（可能还包括少量现金），将其交付给基金公司以换取ETF基金份额。
  - **赎回**：申购的逆向操作，投资者将持有的ETF基金份额交还给基金公司，基金公司则按照 *申购赎回清单*，将等值的一篮子股票（可能也包括少量现金）交付给投资者。
  - ETF 套利
    - 当ETF市价 > 净值时 (溢价套利)：套利者在一级市场申购ETF，在二级市场卖出这些申购来的ETF份额。
    - 当ETF市价 < 净值时 (折价套利)：套利者在二级市场买入便宜的ETF份额，然后立刻在一级市场赎回一篮子股票（可能还包括少量现金）
- `BT_EXECUTE` (期权行权 / Option Exercise)：期权的持有者执行其所拥有的权利
- `BT_QUOTE` (报价 / Quote)：挂单者 Maker 向市场同时报出某个金融产品价格的行为
- `BT_FORQUOTE` (询价 / For-Quote)：投资者向挂单者 Maker 发出一个请求报价的指令。
- `BT_FREEZE` (冻结 / Freeze)： 一种非交易性操作，将账户中的部分资产冻结，使其在一段时间内不能被用于其他普通交易。
- `BT_CREDIT` (融资融券 / Credit Trading)：向券商借钱或借股票来进行交易，从而放大交易能力和潜在盈亏
  - **融资**：借钱买股票（看涨）
  - **融券**：借股卖空（看跌），即借来股票后卖掉，然后过一段时间买入等额的股票归还

### 订单操作类型 `tagActionFlag`
```cpp
/* 定义了对订单可以进行的操作 */
typedef enum tagActionFlag
{
	WAF_CANCEL			= '0',	// 撤销：撤销订单
	WAF_MODIFY			= '3',	// 修改：修改订单
} WTSActionFlag;
```

### 订单状态 `tagOrderState`
```cpp
/* 订单状态枚举，定义了订单在交易过程中的各种状态
 */
typedef enum tagOrderState
{
	WOS_AllTraded				= '0',	// 全部成交：订单已经100%完全成交
	WOS_PartTraded_Queuing,				// 部分成交,仍在队列中：部分成交且仍在排队
	WOS_PartTraded_NotQueuing,			// 部分成交,未在队列：部分成交，订单已经从订单簿移除，但是订单尚未被确认撤销
	WOS_NotTraded_Queuing,				// 未成交：订单进入了订单簿排队等待成交，但目前还没有任何部分成交
	WOS_NotTraded_NotQueuing,			// 未成交,未在队列：零成交，订单已经从订单簿移除，但是订单尚未被确认撤销
	WOS_Canceled,						// 已撤销：订单在完全成交之前，被您主动撤销或因其他规则（如收盘）被系统撤销
	WOS_Submitting				= 'a',	// 正在提交：订单正在提交中，即订单已经由策略生成，并通过交易通道正在发往交易所的途中
	WOS_Cancelling,						// 在撤：订单正在撤销中
	WOS_Nottouched,						// 未触发：专门用于条件单，指的是触发条件还未满足的状态
} WTSOrderState;
```

### 订单类型 `tagOrderType`
```cpp
/* 订单类型枚举 */
typedef enum tagOrderType
{
	WORT_Normal			= 0,		// 正常订单：普通交易订单
	WORT_Exception,					// 异常订单：异常情况下的订单
	WORT_System,					// 系统订单：系统自动生成的订单
	WORT_Hedge						// 对冲订单：对冲交易订单
} WTSOrderType;
```
- `WORT_Normal`：正常订单，由交易策略基于其自身的交易逻辑，主动、正常发出的交易委托
- `WORT_Exception`：异常订单，因为非预期的异常情况而被系统或风控模块强制发出的平仓订单
- `WORT_System`：系统订单，由交易系统本身因为内部管理或结算需要而自动生成的订单，通常与具体策略的交易逻辑无关
- `WORT_Hedge`：对冲订单，专门为了 *对冲（Hedge）* 另一个头寸或资产组合的风险而发出的订单
  - 总是与一个或多个其他头寸相关联，例子：持有某资产多头头寸，但担心未来股市下跌；于是买入一定的空头头寸，这就是一笔对冲订单。

### 成交类型 `tagTradeType`
```cpp
/*
 *	成交类型枚举
 *	定义了成交的不同类型
 */
typedef enum tagTradeType
{
	WTT_Common				= '0',	// 普通：普通成交
	WTT_OptionExecution		= '1',	// 期权执行：期权执行成交
	WTT_OTC					= '2',	// OTC成交：场外成交
	WTT_EFPDerived			= '3',	// 期转现衍生成交：期转现衍生成交
	WTT_CombinationDerived	= '4'	// 组合衍生成交：组合衍生成交
} WTSTradeType;
```
- `WTT_Common`(普通成交 / Common Trade)：通过交易所的集中撮合系统，由买卖双方的普通订单（如限价单、市价单）匹配而产生的标准成交
- `WTT_OptionExecution` (期权执行 / Option Execution)：因为期权被行权而产生的标的资产的成交
- `WTT_OTC` (场外交易成交 / Over-The-Counter Trade)：交易双方在交易所之外私下协商好价格和数量，然后将成交结果报告给交易所进行登记和结算的交易
- `WTT_EFPDerived`(期转现衍生成交 / EFP Derived Trade)：由一笔期转现（EFP）业务所衍生出的期货合约的平仓成交
  - 期货头寸向现货头寸的转换。例子：
    - 某交易者持有一份期货合约（苹果），某现货商（果汁厂）需要一批现货（苹果）；
    - 可以进行一笔 *期转现* 交易：果汁厂接手期货合约，交易者直接获得现货（果汁）或者等值现金
    - 在交易所里，交易者的期货合约需要被平仓，这笔平仓的记录，就被标记为 `WTT_EFPDerived`
- `WTT_CombinationDerived`(组合衍生成交 / Combination Derived Trade)：由一个标准化的组合合约成交后，拆分到其包含的各个单一合约上的成交

### 错误代码类型 `tagErrorCode`
```cpp
typedef enum tagErrorCode
{
	WEC_NONE			=	0,		// 没有错误：操作成功
	WEC_ORDERINSERT,				// 下单错误：下单操作失败
	WEC_ORDERCANCEL,				// 撤单错误：撤单操作失败
	WEC_EXECINSERT,					// 行权指令错误：行权指令操作失败
	WEC_EXECCANCEL,					// 行权撤销错误：行权撤销操作失败
	WEC_UNKNOWN			=	9999	// 未知错误：未识别的错误类型
} WTSErroCode;
```

### 比较字段 `tagCompareField`
```cpp
typedef enum tagCompareField
{
	WCF_NEWPRICE			=	0,	// 最新价：与最新价格比较
	WCF_BIDPRICE,					// 买一价：与买一价格比较
	WCF_ASKPRICE,					// 卖一价：与卖一价格比较
	WCF_PRICEDIFF,					// 价差,止盈止损专用：与价格差值比较
	WCF_NONE				=	9	// 不比较：不进行价格比较
} WTSCompareField;
```

### 比较类型 `tagCompareType`
```cpp
typedef enum tagCompareType
{
	WCT_Equal			= 0,		// 等于：价格等于条件
	WCT_Larger,						// 大于：价格大于条件
	WCT_Smaller,					// 小于：价格小于条件
	WCT_LargerOrEqual,				// 大于等于：价格大于等于条件
	WCT_SmallerOrEqual				// 小于等于：价格小于等于条件
}WTSCompareType;
```

### 行情解析器事件枚举 `tagParserEvent`
```cpp
/* 行情解析器事件枚举，定义了行情解析器可能产生的事件类型 */
typedef enum tagParserEvent
{
	WPE_Connect			= 0,		// 连接事件：连接建立
	WPE_Close,						// 关闭事件：连接关闭
	WPE_Login,						// 登录：登录成功
	WPE_Logout						// 注销：注销登录
}WTSParserEvent;
```

### 交易模块事件枚举 `tagTraderEvent`
```cpp
/* 交易模块事件枚举，定义了交易模块可能产生的事件类型 */
typedef enum tagTraderEvent
{
	WTE_Connect			= 0,		// 连接事件：连接建立
	WTE_Close,						// 关闭事件：连接关闭
	WTE_Login,						// 登录：登录成功
	WTE_Logout						// 注销：注销登录
}WTSTraderEvent;
```

### 交易阶段 `tagTradeStatus`
```cpp
typedef enum tagTradeStatus
{
	TS_BeforeTrading	= '0',	// 开盘前：交易开始前的状态
	TS_NotTrading		= '1',	// 非交易：非交易时段
	TS_Continous		= '2',	// 连续竞价：连续竞价时段
	TS_AuctionOrdering	= '3',	// 集合竞价下单：集合竞价下单时段
	TS_AuctionBalance	= '4',	// 集合竞价平衡：集合竞价平衡时段
	TS_AuctionMatch		= '5',	// 集合竞价撮合：集合竞价撮合时段
	TS_Closed			= '6'	// 收盘：交易结束
}WTSTradeStatus;
```
```mermaid
graph TD;
    A("开始") --> B["TS_BeforeTrading<br/>开盘前准备"];
    B --> C["TS_AuctionOrdering<br/>集合竞价可下单撤单"];
    C --> D["TS_AuctionBalance<br/>集合竞价只可下单"];
    D --> E["TS_AuctionMatch<br/>撮合开盘价"];
    E --> F["TS_Continous<br/>连续交易"];
    F --> G{"有休息（非交易时段）吗?"};
    G -->|是| H["TS_NotTrading<br/>休息"];
    H --> I["TS_Continous<br/>连续交易"];
    G -->|否| I;
    I --> J{"有收盘集合竞价吗?"};
    J -->|是| K["收盘集合竞价流程"];
    K --> L["TS_Closed<br/>收盘"];
    J -->|否| L;
    L --> M("结束");

    style B fill:#f9f,stroke:#333,stroke-width:2px
    style C fill:#bbf,stroke:#333,stroke-width:2px
    style D fill:#bbf,stroke:#333,stroke-width:2px
    style E fill:#bbf,stroke:#333,stroke-width:2px
    style F fill:#9f9,stroke:#333,stroke-width:2px
    style H fill:#f96,stroke:#333,stroke-width:2px
    style I fill:#9f9,stroke:#333,stroke-width:2px
    style K fill:#bbf,stroke:#333,stroke-width:2px
    style L fill:#f66,stroke:#333,stroke-width:2px
```

## 统一错误类 WTSError.hpp
WonderTrader 框架的统一错误类 `WTSError`，包含两个成员
- `WTSErroCode m_errCode`：记录错误类型
- `std::string m_strMsg`：记录错误的详细描述和上下文信息

### 源码
```cpp
class WTSError : public WTSObject
{
protected:
	/**
	 * 保护构造函数
	 * 防止直接实例化，必须通过静态工厂方法创建
	 * 初始化错误代码为WEC_NONE（无错误），错误信息为空字符串
	 */
	WTSError():m_errCode(WEC_NONE),m_strMsg(""){}
	
	/**
	 * 虚析构函数
	 * 支持多态销毁，确保派生类对象能够正确清理资源
	 */
	virtual ~WTSError(){}

public:
	/**
	 * 静态工厂方法：创建错误对象
	 * 
	 * @param ec 错误代码，定义错误的类型（如网络错误、数据错误等）
	 * @param errmsg 错误描述信息，提供错误的详细说明
	 * @return 新创建的错误对象指针，调用者负责管理其生命周期
	 * 
	 * 使用示例：
	 * WTSError* error = WTSError::create(WEC_NETWORK_ERROR, "连接服务器失败");
	 */
	static WTSError* create(WTSErroCode ec, const char* errmsg)
	{
		WTSError* pRet = new WTSError;  // 创建新的错误对象实例
		pRet->m_errCode = ec;           // 设置错误代码
		pRet->m_strMsg = errmsg;        // 设置错误信息字符串

		return pRet;  // 返回创建的实例指针
	}

	const char*		getMessage() const{return m_strMsg.c_str();}
	
	WTSErroCode		getErrorCode() const{return m_errCode;}

protected:
	WTSErroCode		m_errCode;  // 错误代码，标识错误的类型和级别
	std::string		m_strMsg;   // 错误信息，提供错误的详细描述和上下文信息
};
```

## 高性能哈希表和集合 FasterDefs.h
提供高效的哈希 Map 和 Set 类模板，包含
- 封装哈希库 *tsl::robin_map* 和 *ankerl::unordered_dense::map* 而实现的哈希 map 和 set 类模板
    ```cpp
    // 基于 tsl::robin_map 的哈希 Map
    template<class Key, class T>
    class fastest_hashmap : public tsl::robin_map<Key, T> 
    // 值类型为 string 的特化版本
    template<class T>
    class fastest_hashmap<std::string, T> : public tsl::robin_map<std::string, T, string_hash>
    // 基于 tsl::robin_map 的哈希 Set
    template<class Key>
    class fastest_hashset : public tsl::robin_set<Key> 
    // 类型为 string 的特化版本
    template<>
    class fastest_hashset<std::string> : public tsl::robin_set<std::string, string_hash> 

    // 基于 ankerl::unordered_dense::map 的哈希 Map
    template<class Key, class T, class Hash = std::hash<Key>>
    class wt_hashmap : public ankerl::unordered_dense::map<Key, T, Hash> 
    // 值类型为 string 且哈希算法为 string_hash 的特化版本
    template<class T>
    class wt_hashmap<std::string, T, string_hash> : public ankerl::unordered_dense::map<std::string, T, string_hash>
    // 基于 ankerl::unordered_dense::map 的哈希 Set
    template<class Key, class Hash = std::hash<Key>>
    class wt_hashset : public ankerl::unordered_dense::set<Key, Hash> 
    // 类型为 string 且哈希算法为 string_hash 的特化版本
    template<>
    class wt_hashset<std::string, string_hash> : public ankerl::unordered_dense::set<std::string, string_hash> 
    ```
- 基于 *BKDRHash 算法* 的字符串哈希算法 
    ```cpp
    struct string_hash
    {
        std::size_t operator()(const std::string& key) const
        {
            size_t seed = 131; // 31 131 1313 13131 131313 etc..  // 设置BKDRHash算法的种子值
            size_t hash = 0;  // 初始化哈希值为0

            char* str = (char*)key.c_str();  // 获取字符串的C风格指针
            while (*str)  // 遍历字符串的每个字符
            {
                hash = hash * seed + (*str++);  // 执行BKDRHash算法：hash = hash * seed + char
            }

            return (hash & 0x7FFFFFFF);  // 返回31位正整数结果，去掉符号位
        }
    };
    ```

## 商品和合约结构体 LoaderDef.hpp

### 商品结构体 `_Contract` 及其映射类型 `ContractMap`
```cpp
/**
 * @brief 合约信息结构体
 * 定义了单个合约的详细信息，包括合约代码、交易所、名称、产品类型等。
 * 还包含交易限制、保证金比例、期权特性等交易相关的参数。
 */
typedef struct _Contract
{
	std::string	m_strCode;           // 合约代码
	std::string	m_strExchg;          // 交易所代码
	std::string	m_strName;           // 合约名称
	std::string	m_strProduct;        // 产品类型

	uint32_t	m_maxMktQty;         // 最大市价单数量
	uint32_t	m_maxLmtQty;         // 最大限价单数量
	uint32_t	m_minMktQty;         // 最小市价单数量
	uint32_t	m_minLmtQty;         // 最小限价单数量

	uint32_t	m_uOpenDate;         // 上市日期
	uint32_t	m_uExpireDate;       // 到期日期

	double		m_dLongMarginRatio;  // 多头保证金比例
	double		m_dShortMarginRatio; // 空头保证金比例

	OptionType	m_optType;           // 期权类型
	std::string m_strUnderlying;     // 标的合约代码
	double		m_strikePrice;       // 行权价格
	double		m_dUnderlyingScale;  // 标的比例
} Contract;

/* 合约信息映射类型
 * 使用合约描述字符串作为键，存储合约信息对象的映射容器。
 */
typedef std::map<std::string, Contract> ContractMap;
```

### 合约结构体 `_Commodity` 及其映射类型 `CommodityMap`
```cpp
/**
 * @brief 商品信息结构体
 * 定义了商品的基本属性，包括名称、交易所、产品类型、货币等。
 * 还包含交易规则、价格精度、交易模式等交易相关的参数。
 */
typedef struct _Commodity
{
	std::string	m_strName;           // 商品名称
	std::string	m_strExchg;          // 交易所代码
	std::string	m_strProduct;        // 产品类型
	std::string	m_strCurrency;       // 货币类型
	std::string m_strSession;        // 交易时间模板

	uint32_t	m_uVolScale;         // 数量比例
	double		m_fPriceTick;        // 价格变动单位
	uint32_t	m_uPrecision;        // 价格精度

	ContractCategory	m_ccCategory;    // 合约类别
	CoverMode			m_coverMode;    // 平仓模式
	PriceMode			m_priceMode;    // 价格模式
	TradingMode			m_tradeMode;    // 交易模式

} Commodity;

/* 商品信息映射类型
 * 使用商品描述字符串作为键，存储商品信息对象的映射容器。
 */
typedef std::map<std::string, Commodity> CommodityMap;
```

# 核心数据层

## 基础对象 WTSObject.hpp

### 虚函数
当基类 Father 和子类 Child 有同一方法时 f，考虑一个实际指向子类对象的父类指针
```cpp
Father* p = new Child();
p->f();
```
- 这里实际调用的是父类的 f 方法，因为编译器采用的是 ***静态绑定***：编译器只看指针的类型，完全不关心这个指针在程序运行时实际指向的是什么对象

当给基类的方法添加关键字 **virtual**，就实现了 ***动态绑定***
- virtual 告诉编译器：“这个函数的行为可能会在子类中改变，不要在编译时就决定调用哪个版本，请在程序运行时，根据指针实际指向的对象来决定” 

### 提供引用计数和自动内存管理的基础对象类 `WTSObject`
对于在堆上创建的对象（通过 new），离开作用域时系统并不会自动销毁，而是需要使用 delete 手动销毁。
- 如果 new 了一个对象但是忘记销毁，那么会产生 ***内存泄漏***
- 如果销毁了一个堆上的对象，但是还有指针指向原来的地址，该指针即成为 ***悬垂指针（Dangling Pointer）***。

`WTSObject` 的设计思想：作为 WonderTrader 中所有数据类型的基类，例如对于类型 A，其继承了 `WTSObject`
- 创建类型 A 的对象 obj：`WTSObject* obj = new MyObject();`（引用计数为 1）
- 在需要对 obj 建立指针/引用的代码块之前：`obj->retain();`（引用计数加 1）
- 对应的在使用完之后：`obj->release()`（引用计数减 1）
  - 当引用计数减至 0，会自动对 obj 调用虚析构

#### 源码
```cpp
class WTSObject
{
public:
	/* 构造函数，初始化引用计数为1 */
	WTSObject() :m_uRefs(1){}
	
	/* 虚析构函数，支持多态销毁，确保派生类对象能够正确清理资源 */
	virtual ~WTSObject(){}

public:
	/* 增加引用计数 */
	inline uint32_t		retain(){ return m_uRefs.fetch_add(1) + 1; }

	/*减少引用计数
	 * 当不再需要对象时调用，引用计数为0时自动销毁对象
	 * 使用异常保护确保即使在异常情况下也能正确处理
	 */
	virtual void	release()
	{
		if (m_uRefs == 0)  // 防御性检查：如果引用计数已经为0，直接返回
			return;

		try
		{
			uint32_t cnt = m_uRefs.fetch_sub(1);  // 原子操作减少引用计数
			if (cnt == 1)  // 如果减少前引用计数为1，说明这是最后一个引用
			{
				delete this;  // 销毁当前对象，会调用虚析构函数
			}
		}
		catch(...)  // 捕获所有异常，确保release操作不会抛出异常
		{
			// 静默处理异常，保证程序稳定性
			// 在实际应用中可以考虑记录日志
		}
	}

	/* 检查是否只有一个引用
	 * @return 如果引用计数为1返回true，否则返回false
	 */
	inline bool			isSingleRefs() { return m_uRefs == 1; }

	/* 获取当前引用计数
	 * @return 当前的引用计数值
	 */
	inline uint32_t		retainCount() { return m_uRefs; }

protected:
	volatile std::atomic<uint32_t>	m_uRefs;  // 原子引用计数器，使用volatile确保内存可见性
};
```

### 基础对象内存池模板类 `WTSPoolObject`
基础对象类 `WTSObject` 只是为了方便预防 *内存泄漏* 和 *悬垂指针* 的问题。
当对于某个类型 A，需要频繁、大量的创建、销毁对象时，相较而言 `new` 和 `delete` 的开销就比较大了。

基础对象内存池模板类 `WTSPoolObject` 的设计思想：
- 将一个类型 A *基础且池化* 是指
	```cpp
	class A : public WTSPoolObject<A>
	```
  - 注意！此时 A <—— WTSPoolObject\<A\> <—— WTSObject，所以对 A 的对象建立指针/引用时，仍需使用 WTSObject 的 retain/releasse
- 这样类型 A 就可以通过静态方法 `allocate` 来以内存池方式创建对象
	```cpp
	A* a = A::allocate();
	```
- 归还对象对应为
	```cpp
	a->release();
	```

#### 源码
```cpp
template<typename T>
class WTSPoolObject : public WTSObject
{
private:
	typedef ObjectPool<T> MyPool;  // 类型别名：定义当前类型的对象池类型
	MyPool*		_pool;             // 对象池指针，指向创建此对象的对象池
	SpinMutex*	_mutex;            // 自旋互斥锁指针，用于保护对象池操作

public:
	WTSPoolObject():_pool(NULL){}
	
	virtual ~WTSPoolObject() {}

public:
	static T*	allocate()
	{
		/*
		 * 线程局部存储的对象池和锁
		 * 每个线程拥有独立的对象池实例，减少线程间竞争
		 */
		thread_local static MyPool		pool;   // 线程局部对象池
		thread_local static SpinMutex	mtx;    // 线程局部自旋锁

		mtx.lock();                    // 加锁保护对象池操作
		T* ret = pool.construct();     // 从对象池中构造对象
		mtx.unlock();                  // 解锁
		
		ret->_pool = &pool;            // 设置对象的池指针
		ret->_mutex = &mtx;            // 设置对象的锁指针
		return ret;                    // 返回构造的对象
	}

public:
	virtual void release() override
	{
		if (m_uRefs == 0)  // 防御性检查：引用计数已为0
			return;

		try
		{
			uint32_t cnt = m_uRefs.fetch_sub(1);  // 原子操作减少引用计数
			if (cnt == 1)  // 如果这是最后一个引用
			{
				_mutex->lock();                    // 加锁保护对象池操作
				_pool->destroy((T*)this);         // 将对象回收到对象池（实际是重置状态）
				_mutex->unlock();                  // 解锁
			}
		}
		catch (...)  // 异常保护，确保release操作的安全性
		{
			// 静默处理异常，保证程序稳定性
		}
	}
};
```

## 变体数据类型 WTSVariant.hpp

## 核心数据结构 WTSStruct.h

## 数据集合 WTSCollection.hpp